# Day 2 Of Challange
### Learn:

- Spark architecture (driver, executors, DAG)
- DataFrames vs RDDs
- Lazy evaluation
- Notebook magic commands (`%sql`, `%python`, `%fs`)

### 🛠️ Tasks:

1. Upload sample e-commerce CSV
2. Read data into DataFrame
3. Perform basic operations: select, filter, groupBy, orderBy
4. Export results

###importing necessary functions


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

### Taking Sample 100 Records from the complete data Set

In [0]:
df = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv", header=True, inferSchema=True)
df.show(100)

+-------------------+----------+----------+-------------------+--------------------+----------------+-------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|           brand|  price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+----------------+-------+---------+--------------------+
|2019-10-01 00:00:00|      view|  44600062|2103807459595387724|                NULL|        shiseido|  35.79|541312140|72d76fde-8bb3-4e0...|
|2019-10-01 00:00:00|      view|   3900821|2053013552326770905|appliances.enviro...|            aqua|   33.2|554748717|9333dfbd-b87a-470...|
|2019-10-01 00:00:01|      view|  17200506|2053013559792632471|furniture.living_...|            NULL|  543.1|519107250|566511c2-e2e3-422...|
|2019-10-01 00:00:01|      view|   1307067|2053013558920217191|  computers.notebook|          lenovo| 251.74|550050854|7c90fc70-0e80-459...|
|2019-10-01 0

In [0]:
events = df.limit(100)

In [0]:
events.count()

100

In [0]:
events.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



### Doing Select Operation on the Event Data Set

In [0]:
events.select("event_type","product_id","price").show(20)

+----------+----------+-------+
|event_type|product_id|  price|
+----------+----------+-------+
|      view|  44600062|  35.79|
|      view|   3900821|   33.2|
|      view|  17200506|  543.1|
|      view|   1307067| 251.74|
|      view|   1004237|1081.98|
|      view|   1480613| 908.62|
|      view|  17300353| 380.96|
|      view|  31500053|  41.16|
|      view|  28719074| 102.71|
|      view|   1004545| 566.01|
|      view|   2900536|  51.46|
|      view|   1005011| 900.64|
|      view|   3900746| 102.38|
|      view|  44600062|  35.79|
|      view|  13500240|  93.18|
|      view|  23100006| 357.79|
|      view|   1801995| 193.03|
|      view|  10900029|  58.95|
|      view|   1306631| 580.89|
|      view|   1005135|1747.79|
+----------+----------+-------+
only showing top 20 rows


### Finding Distinct Brand in the Data set

In [0]:
events.select("brand").distinct().show()

+--------+
|   brand|
+--------+
|shiseido|
|    aqua|
|    NULL|
|  lenovo|
|   apple|
|  pulser|
|   creed|
|luminarc|
|   baden|
|  huawei|
|elenberg|
| samsung|
|   haier|
|     brw|
|   bosch|
|      hp|
|     jbl|
|  xiaomi|
|   midea|
| ariston|
+--------+
only showing top 20 rows


###Using Filters

In [0]:
events.filter(events["category_code"].isNull()).count()

34

In [0]:
events.filter(events.brand == "haier").show()

+-------------------+----------+----------+-------------------+--------------------+-----+------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|brand| price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+-----+------+---------+--------------------+
|2019-10-01 00:00:13|      view|   3900746|2053013552326770905|appliances.enviro...|haier|102.38|555444559|98b88fa0-d8fa-4b9...|
|2019-10-01 00:00:18|      view|   1801995|2053013554415534427|electronics.video.tv|haier|193.03|537192226|e3151795-c355-4ef...|
|2019-10-01 00:00:31|      view|   3900746|2053013552326770905|appliances.enviro...|haier|102.38|555444559|98b88fa0-d8fa-4b9...|
+-------------------+----------+----------+-------------------+--------------------+-----+------+---------+--------------------+



In [0]:
events.filter(events.brand == "lenovo").show()

+-------------------+----------+----------+-------------------+------------------+------+------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|     category_code| brand| price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+------------------+------+------+---------+--------------------+
|2019-10-01 00:00:01|      view|   1307067|2053013558920217191|computers.notebook|lenovo|251.74|550050854|7c90fc70-0e80-459...|
|2019-10-01 00:01:06|      view|   1307004|2053013558920217191|computers.notebook|lenovo|290.61|542378517|244570b9-ebb4-4d4...|
+-------------------+----------+----------+-------------------+------------------+------+------+---------+--------------------+



###Using Group by

In [0]:
result = events.groupBy("brand").agg(sum("price").alias("total_price")).orderBy(col("total_price").desc())
display(result)

brand,total_price
apple,8017.68
samsung,6366.240000000001
null,2809.92
hp,2414.02
pulser,1830.1100000000001
sony,872.01
huawei,844.56
lucente,715.3399999999999
lenovo,542.35
acer,514.79


In [0]:
events.withColumn("event_time", to_date(events["event_time"])).select("event_time").distinct().show()

+----------+
|event_time|
+----------+
|2019-10-01|
+----------+



In [0]:
events.groupBy("user_id").count().filter(col("count") >=3 ).show()



+---------+-----+
|  user_id|count|
+---------+-----+
|554748717|    3|
|519107250|    3|
|550050854|    3|
|535871217|    3|
|550978835|    3|
|555444559|    3|
|555446365|    3|
|513642368|    3|
|555217733|    3|
|512558158|    4|
+---------+-----+



In [0]:
from pyspark.sql.functions import sum, col

max_brand = events.groupBy("brand").agg(sum("price").alias("total_price")) \
    .orderBy(col("total_price").desc()) \
    .limit(1)

display(max_brand)

brand,total_price
apple,8017.68


In [0]:
top_brands = events.groupBy("brand").count().orderBy("count", ascending=False)
display(top_brands)

brand,count
samsung,12
null,11
apple,8
haier,3
hp,3
redmond,3
respect,3
lucente,3
sony,3
shiseido,2
